In [2]:
import dask.bag as db
import json
from dask.distributed import Client
import dask.dataframe as dd
import xarray as xr

In [ ]:
client = Client()
client.dashboard_link

In [ ]:
bag = db.read_text("data/ping-2025-06-10T0000.bz2", blocksize=None).map(json.loads)

In [ ]:
bag.take(2)

In [5]:
bag2 = bag.map(lambda rec: {**rec, "result": json.dumps(rec["result"])})

In [4]:
# types = bag.map(lambda rec: type(rec.get("ttl"))).distinct().compute()
# print(types)

In [ ]:
# 1. Pull a representative sample (e.g. first 1 000 records)
sample = bag2.take(1000, warn=False)

# 2. Build a pandas DataFrame from the sample and drop all rows
import pandas as pd
meta = pd.DataFrame(sample).head(1)
display(meta)
print()
# 3. Inspect `meta` to confirm column names and dtypes
meta["ttl"] = meta["ttl"].astype("Int64")
print(meta.dtypes)

In [ ]:
def parse_ping(record):
    try:
        return {
            'src_addr': record['src_addr'],
            'dst_addr': record['dst_addr'],
            'timestamp': record['timestamp'],
            'rtt': record['avg']
        }
    except (KeyError, IndexError):
        return 'k'

In [29]:
df = bag2.to_dataframe(meta=meta)

In [30]:
df.to_parquet("data/ping-2025-06-10-0h-lat", engine="pyarrow")